In [1]:
import pandas as pd
import os
import graph_tool.all as gt
import h3
import pyproj
geod = pyproj.Geod(ellps="WGS84")

In [2]:
path = 'gtfs_rio-de-janeiro'
# Cargar GTFS
stops = pd.read_csv(os.path.join(path, "stops.txt"))
stop_times = pd.read_csv(os.path.join(path, "stop_times.txt"))
trips = pd.read_csv(os.path.join(path,"trips.txt"))
routes = pd.read_csv(os.path.join(path,"routes.txt"))

/tmp/ipykernel_227833/526264406.py:4: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  stop_times = pd.read_csv(os.path.join(path, "stop_times.txt"))


In [3]:
# 1. Secuencia de stops por trip
stop_sequences = stop_times.sort_values(['trip_id', 'stop_sequence']) \
    .groupby('trip_id')['stop_id'].apply(tuple).reset_index()

# 2. Añadir info de ruta
stop_sequences = stop_sequences.merge(trips.loc[trips.direction_id == 0,['trip_id', 'route_id']], on='trip_id')

# 3. Quitar duplicados por (route_id, secuencia)
unique_sequences = stop_sequences.drop_duplicates(subset=['route_id', 'stop_id'])

# Generar enlaces consecutivos
edges = []
for trip_id, group in unique_sequences.groupby('trip_id'):
    group = group.reset_index(drop=True)
    route_id = group.route_id.values[0]
    nodes = group.stop_id.values[0]
    if len(nodes) > 2:
        for i, j in zip(nodes[:-1], nodes[1:]):
            edges.append((i, j, route_id, trip_id))

edges_df = pd.DataFrame(edges, columns=['nodo_i', 'nodo_j', 'name', 'trip'])

# Opcional: asociar nombres de paradas y rutas
edges_df = edges_df.merge(stops[['stop_id', 'stop_name']], left_on='nodo_i', right_on='stop_id', how='left') \
                   .rename(columns={'stop_name': 'nombre_i'}) \
                   .drop(columns=['stop_id'])

edges_df = edges_df.merge(stops[['stop_id', 'stop_name']], left_on='nodo_j', right_on='stop_id', how='left') \
                   .rename(columns={'stop_name': 'nombre_j'}) \
                   .drop(columns=['stop_id'])

edges_df = edges_df.merge(routes[['route_id', 'route_long_name']], left_on='name', right_on='route_id', how='left') \
                   .drop(columns=['route_id'])



In [4]:
res = 10
stops.loc[:,'cell'] = stops.apply(lambda x: h3.latlng_to_cell(x.stop_lat, x.stop_lon,res), axis = 1)

edges_df.loc[:,'cell_i'] = edges_df.merge(stops[['stop_id','cell']], left_on = 'nodo_i', right_on = 'stop_id').cell
edges_df.loc[:,'cell_j'] = edges_df.merge(stops[['stop_id','cell']], left_on = 'nodo_j', right_on = 'stop_id').cell

In [5]:
nodes2int = pd.Series({n : i for n, i in enumerate(set(edges_df.cell_i).union(set(edges_df.cell_j)))}).reset_index()
nodes2int.rename({'index' : 'node_int', 0 : 'node_id'}, axis = 1, inplace=True)


edges_df.loc[:,'int_i'] = edges_df.merge(nodes2int, left_on = 'cell_i', right_on = 'node_id').node_int
edges_df.loc[:,'int_j'] = edges_df.merge(nodes2int, left_on = 'cell_j', right_on = 'node_id').node_int

In [6]:
g = gt.Graph(edges_df[['int_i', 'int_j', 'trip']].values.tolist(), eprops = [('name','object')], directed=False)
gt.remove_self_loops(g)

In [7]:
pos = g.new_vertex_property('vector<double>')
ids = g.new_vertex_property('object')
for v in g.vertices():
    lat, lon = h3.cell_to_latlng(nodes2int.loc[nodes2int.node_int == v].node_id.values[0])
    pos[v] = (lon, -lat)
    ids[v] = nodes2int.loc[nodes2int.node_int == v].node_id.values[0]
g.vp.pos = pos
g.vp.ids = ids
weight = g.new_edge_property('double')
for e in g.edges():
    u = e.source()
    v = e.target()
    _,_, dist = geod.inv(pos[u][0],
                         -pos[u][1],
                         pos[v][0],
                         -pos[v][1]
                         )
    weight[e] = dist
g.ep.weight = weight


In [8]:
g.save('redes/redRIO.gt')